##### 0) 라이브러리 불러오기

In [1]:
# (1) 기본 라이브러리
import pandas as pd
import numpy as np

# (2) 머신러닝 라이브러리
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn.functional as F

##### 1) 데이터 불러오기

In [2]:
df = pd.read_csv("AWE_1000_score.csv")

In [3]:
df.head(2)

,Unnamed: 0,id,nationality,sex,title,content,score
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70


##### 2) 문장 개수 추출

In [4]:
df["sentence"] = df["content"].apply(lambda x: len(x.split("."))) # !!!
#df["content"].apply(lambda x: len(x.split("."))).value_counts()

##### 3) Corpus 추출

In [5]:
corpus = df["content"]

##### 4) TF-IDF 추출

In [6]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [7]:
len(vectorizer.get_feature_names_out())

8093

In [8]:
X

<1000x8093 sparse matrix of type '<class 'numpy.float64'>'
	with 34048 stored elements in Compressed Sparse Row format>

##### 5) Content Vector 추출

In [9]:
df["content_vector"] = X.todense().tolist() # !!!

In [10]:
df["content_vector"]

0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                             ...                        
995    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
996    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
997    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
998    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1549718154414...
999    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: content_vector, Length: 1000, dtype: object

In [11]:
df.head(2)

,Unnamed: 0,id,nationality,sex,title,content,score,sentence,content_vector
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80,18,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


##### 6) 모델 정의

In [89]:
class MultivariableLinearModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = torch.nn.Linear(in_features=8093, out_features= 1)

  def forward(self, x):
    return self.linear(x)

myLin = MultivariableLinearModel()

##### 7) Optimizer 정의

In [91]:
optimizer = torch.optim.SGD(myLin.parameters(), lr=0.01)

##### 8) 모델 학습

In [100]:
# (1) Epoch 순회
nb_epochs = 10
for epoch in range(nb_epochs + 1):
  total_loss = 0
  for idx in range(len(df)):
    # 1] 학습 데이터 불러오기
    x_train = np.array([df["content_vector"][idx]])
    y_train = np.array([df["score"][idx]])
    x_train = torch.from_numpy(x_train).float()
    y_train = torch.from_numpy(y_train).int()
    # 2] 모델 예측
    hypothesis = myLin(x_train)
    # 3] Cost 계산
    cost = F.mse_loss(hypothesis.to(torch.float32), y_train.to(torch.float32))
    # 4] Loss 계산
    total_loss = total_loss + cost.item()
    # 5] 모델 역전파
    optimizer.zero_grad() 
    cost.backward() 
    optimizer.step()
  
  # 6] 결과 출력
  print(epoch, total_loss / len(df))

<ipython-input-100-0cdedf403e2c>:11: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = F.mse_loss(hypothesis.to(torch.float32), y_train.to(torch.float32))


0 175.76021923838323
1 167.82148048308608
2 160.50617327039515
3 153.741971000402
4 147.4671060589794
5 141.6285951287909
6 136.18078219308686
7 131.08420806209884
8 126.3045435849356
9 121.81189155275398
10 117.58005926325976
